In [286]:
import numpy as np
import random
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import train_test_split 

In [287]:
df = pd.read_csv('train.csv')

In [288]:
def preProcess(features):
    for i, d in enumerate(features):
        mini = min(d)
        maxi = max(d)-mini
        for ii, dd in enumerate(d):
            tmp = (dd-mini)/maxi
            if tmp==1:
                features[i][ii] = 0.99
            elif tmp==0:
                features[i][ii] = 0.01
            else:
                features[i][ii] = tmp

In [289]:
CreditScore = df['CreditScore'].tolist()
Geography = df['Geography'].tolist()
Gender = df['Gender'].tolist()
Age = df['Age'].tolist()
Tenure = df['Tenure'].tolist()
Balance = df['Balance'].tolist()
NumOfProducts = df['NumOfProducts'].tolist()
HasCrCard = df['HasCrCard'].tolist()
IsActiveMember = df['IsActiveMember'].tolist()
EstimatedSalary = df['EstimatedSalary'].tolist()
Exited = df['Exited'].tolist()

features = (CreditScore, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary)
preProcess(features)        

for i, d in enumerate(Geography):
    if d=='S0':
        Geography[i] = 0.01
    elif d=='S1':
        Geography[i] = 0.5
    else:
        Geography[i] = 0.99
        
for i, d in enumerate(Gender):
    if d=='Male':
        Gender[i] = 0.99
    else:
        Gender[i] = 0.01

In [290]:
X = []
for _ind in range(len(Exited)):
    X.append((CreditScore[_ind], Geography[_ind], Gender[_ind], Age[_ind], Tenure[_ind], Balance[_ind],\
                   NumOfProducts[_ind], HasCrCard[_ind], IsActiveMember[_ind], EstimatedSalary[_ind]))
y = Exited.copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) 

In [294]:
sess = tf.Session()

num_x = 10
batch_size = 128
lr_rate = 0.0001
gamma_val = -25.0
num_epoch = 64

x_data = tf.placeholder(shape=[None, num_x], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
prediction_grid = tf.placeholder(shape=[None, num_x], dtype=tf.float32)
b = tf.Variable(tf.random_normal(shape=[1,batch_size]))

In [295]:
gamma = tf.constant(gamma_val)
dist = tf.reduce_sum(tf.square(x_data), 1)
dist = tf.reshape(dist, [-1,1])
sq_dists = tf.add(tf.subtract(dist, tf.multiply(float(num_x), tf.matmul(x_data,tf.transpose(x_data)))), tf.transpose(dist))
my_kernel = tf.exp(tf.multiply(gamma, tf.abs(sq_dists)))

model_output = tf.matmul(b, my_kernel)
first_term = tf.reduce_sum(b)
b_vec_cross = tf.matmul(tf.transpose(b), b)
y_target_cross = tf.matmul(y_target, tf.transpose(y_target))
second_term = tf.reduce_sum(tf.multiply(my_kernel, tf.multiply(b_vec_cross,y_target_cross)))
loss = tf.negative(tf.subtract(first_term, second_term))

rA = tf.reshape(tf.reduce_sum(tf.square(x_data), 1),[-1,1])
rB = tf.reshape(tf.reduce_sum(tf.square(prediction_grid), 1),[-1,1])
pred_sq_dist = tf.add(tf.subtract(rA, tf.multiply(float(num_x), tf.matmul(x_data,
tf.transpose(prediction_grid)))), tf.transpose(rB))
pred_kernel = tf.exp(tf.multiply(gamma, tf.abs(pred_sq_dist)))
prediction_output = tf.matmul(tf.multiply(tf.transpose(y_target),b),pred_kernel)
prediction = tf.sign(prediction_output-tf.reduce_mean(prediction_output))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.squeeze(prediction),tf.squeeze(y_target)), tf.float32))

my_opt = tf.train.GradientDescentOptimizer(lr_rate)
train_step = my_opt.minimize(loss)
init = tf.initialize_all_variables()
sess.run(init)

In [296]:
loss_vec = []
batch_accuracy = []
x_vals = np.array(X_train)
y_vals = np.array(y_train)

rand_index = list(range(len(x_vals)))
pt = 0
while(num_epoch):
    if pt+batch_size >= len(x_vals):
        random.shuffle(rand_index)
        num_epoch-=1
        pt = 0
    X = x_vals[rand_index[pt:pt+batch_size]]
    Y = np.transpose([y_vals[rand_index[pt:pt+batch_size]]])
    pt+=batch_size
    sess.run(train_step, feed_dict={x_data: X, y_target:Y})
    temp_loss = sess.run(loss, feed_dict={x_data: X, y_target: Y})
    loss_vec.append(temp_loss)
    acc_temp = sess.run(accuracy, feed_dict={x_data: X,y_target: Y,prediction_grid:X})
    batch_accuracy.append(acc_temp)


In [297]:
batch_accuracy

[0.21875,
 0.0,
 0.328125,
 0.0,
 0.1484375,
 0.015625,
 0.0,
 0.28125,
 0.0,
 0.265625,
 0.2109375,
 0.1875,
 0.0,
 0.203125,
 0.25,
 0.203125,
 0.0078125,
 0.1796875,
 0.2265625,
 0.0,
 0.0,
 0.0,
 0.1953125,
 0.03125,
 0.1953125,
 0.0,
 0.0,
 0.015625,
 0.1015625,
 0.0,
 0.2109375,
 0.015625,
 0.0,
 0.21875,
 0.203125,
 0.1796875,
 0.0,
 0.0,
 0.0,
 0.1796875,
 0.1875,
 0.203125,
 0.0078125,
 0.21875,
 0.1484375,
 0.0078125,
 0.0,
 0.21875,
 0.234375,
 0.0,
 0.1953125,
 0.1953125,
 0.0078125,
 0.015625,
 0.25,
 0.015625,
 0.2109375,
 0.0,
 0.0,
 0.015625,
 0.0,
 0.015625,
 0.0,
 0.0,
 0.0,
 0.015625,
 0.0,
 0.0,
 0.171875,
 0.0,
 0.015625,
 0.0,
 0.2734375,
 0.0,
 0.203125,
 0.0,
 0.0,
 0.2578125,
 0.0,
 0.2265625,
 0.0,
 0.1640625,
 0.2734375,
 0.0,
 0.203125,
 0.0,
 0.15625,
 0.2265625,
 0.0,
 0.0,
 0.1875,
 0.15625,
 0.203125,
 0.0,
 0.0,
 0.1796875,
 0.203125,
 0.203125,
 0.0,
 0.015625,
 0.0,
 0.0,
 0.0,
 0.234375,
 0.2109375,
 0.15625,
 0.1875,
 0.0078125,
 0.0,
 0.0,
 0.0,
 0

In [298]:
loss_vec

[-9.241484,
 -9.254284,
 -9.267086,
 -9.279885,
 -9.292686,
 -9.173876,
 -9.31822,
 -10.170552,
 -9.343773,
 -9.356571,
 -9.369374,
 -9.834585,
 -9.394661,
 -9.407821,
 -9.297552,
 -9.487842,
 -9.520709,
 -9.438246,
 -8.906161,
 -9.484908,
 -9.497511,
 -9.510506,
 -9.523308,
 -9.143724,
 -9.54873,
 -9.561528,
 -9.574331,
 -9.548695,
 -9.599913,
 -9.612722,
 -9.6255245,
 -9.818538,
 -9.651032,
 -10.134649,
 -10.440563,
 -9.68951,
 -9.702311,
 -9.715109,
 -9.727913,
 -9.740713,
 -9.69853,
 -9.76633,
 -9.792785,
 -9.791932,
 -9.804729,
 -10.508267,
 -9.830544,
 -9.843062,
 -9.855883,
 -9.868684,
 -10.028875,
 -9.8943815,
 -9.907298,
 -9.909992,
 -9.93277,
 -9.934597,
 -9.937086,
 -9.971192,
 -9.9839945,
 -9.972429,
 -10.009579,
 -10.309216,
 -10.035184,
 -10.047983,
 -10.060785,
 -10.055718,
 -10.086369,
 -10.09917,
 -10.1053295,
 -10.124787,
 -10.09816,
 -10.15036,
 -10.16285,
 -10.175959,
 -10.188762,
 -10.201657,
 -10.214361,
 -10.2271595,
 -10.239963,
 -10.252762,
 -10.265562,
 -10.27